# Gold Stage ! Star Schema & optimized

# fact Trip
## Merge the trip, dimDate (twice), and dimRiders data frames based on the required columns. Here is an example:

In [ ]:
merged_df = pd.merge(trip_df[['trip_id', 'rideable_type', 'start_at', 'ended_at', 'start_station_id', 'end_station_id', 'rider_id']],
                     dim_date_df[['DateTime']],
                     left_on='start_at', right_on='DateTime', how='inner')
merged_df = pd.merge(merged_df, dim_date_df[['DateTime']],
                     left_on='ended_at', right_on='DateTime', how='inner')
merged_df = pd.merge(merged_df, dim_riders_df[['rider_key', 'birthday']],
                     left_on='rider_id', right_on='rider_key', how='inner')

## Convert the 'start_at' and 'ended_at' columns to datetime objects, and create a new column for age:

In [ ]:
merged_df['start_at'] = pd.to_datetime(merged_df['start_at'])
merged_df['ended_at'] = pd.to_datetime(merged_df['ended_at'])
merged_df['age'] = (merged_df['start_at'] - pd.to_datetime(merged_df['birthday'])).astype('<m8[Y]')


In [ ]:
ffrom pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from datetime import datetime

# Define a UDF to calculate age from birth year
def calculate_age(dob, started_at):
    anniversary_date = datetime(started_at.year, dob.month, dob.day)
    age = started_at.year - dob.year - ((started_at < anniversary_date))
    return age

# Register the UDF
calculate_age_udf = udf(lambda dob, started_at: calculate_age(dob, started_at), IntegerType())

# Apply the UDF to the dob column and store the result in a new column called age
df = df.withColumn('age', calculate_age_udf(df.dob, df.started_at))



# Create a new column for trip duration:

In [ ]:
merged_df['trip_duration'] = merged_df['ended_at'] - merged_df['start_at']


# Rename the 'DateTime' columns to 'start_date_id' and 'ended_date_id', and select the required columns:


In [ ]:
merged_df = merged_df.rename(columns={'DateTime_x': 'start_date_id', 'DateTime_y': 'ended_date_id'})
fact_trip_df = merged_df[['trip_id', 'rideable_type', 'start_date_id', 'ended_date_id', 'start_station_id', 'end_station_id', 'rider_id', 'age', 'trip_duration']]


# Set 'trip_id' as the index of the new data frame:

In [ ]:
fact_trip_df.set_index('trip_id', inplace=True)


# Riders DF create

In [ ]:
data = {
    'rider_key': [],
    'first_name': [],
    'last_name': [],
    'address': [],
    'birthday': [],
    'account_start_date': [],
    'account_end_date': [],
    'is_member': []
}

df = pd.DataFrame(data)
df['rider_key'] = df['rider_key'].astype('int32')
df['first_name'] = df['first_name'].astype('str')
df['last_name'] = df['last_name'].astype('str')
df['address'] = df['address'].astype('str')
df['birthday'] = pd.to_datetime(df['birthday'])
df['account_start_date'] = pd.to_datetime(df['account_start_date'])
df['account_end_date'] = pd.to_datetime(df['account_end_date'])
df['is_member'] = df['is_member'].astype('bool')
df.set_index('rider_key', inplace=True)
